# Automatische Klassifikation von Support-Tickets (Plotly-Version)

In diesem Notebook nutzen wir EML-Dateien aus den Verzeichnissen `data/training_data` und `data/test_data` für Training und Testing.
Es werden sowohl Zero-Shot- als auch Few-Shot-Ansätze demonstriert und wir verwenden Plotly, um die DataFrames darzustellen.


## 1. Installation und Importe

Stellt sicher, dass ihr die benötigten Bibliotheken installiert habt (`pip install openai pandas plotly`).


In [2]:
# Bibliotheken importieren
import os
import glob
import pandas as pd
from email import policy
from email.parser import BytesParser
import openai  # Für LLM-Abfragen
import plotly.graph_objects as go
from dotenv import load_dotenv  # Ergänzt für .env-Unterstützung

# .env-Datei laden (API-Key wird als Umgebungsvariable verfügbar)
load_dotenv()

# Funktion, um DataFrames mit Plotly anzuzeigen
def display_df(df: pd.DataFrame, title: str):
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns), fill_color='lightgrey', align='left'),
        cells=dict(values=[df[col] for col in df.columns], align='left')
    )])
    fig.update_layout(title=title)
    fig.show()

# OpenAI API-Key sicher aus Umgebungsvariable lesen
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY nicht gefunden. Bitte in der .env-Datei setzen.")
client = openai.OpenAI(api_key=api_key)

## 2. Datenverzeichnis und Struktur

- Trainingsdaten (EML-Dateien) liegen in `data/training_data`
- Testdaten (EML-Dateien) liegen in `data/test_data`


In [3]:
# Verzeichnisse definieren
train_dir = "data/training_data"
test_dir = "data/test_data"

# Existenz prüfen
if not os.path.isdir(train_dir):
    raise FileNotFoundError(f"Trainingsverzeichnis '{train_dir}' nicht gefunden")
if not os.path.isdir(test_dir):
    raise FileNotFoundError(f"Testverzeichnis '{test_dir}' nicht gefunden")

print(f"Trainingsverzeichnis: {train_dir}")
print(f"Testverzeichnis: {test_dir}")

Trainingsverzeichnis: data/training_data
Testverzeichnis: data/test_data


## 3. Einlesen der EML-Dateien

Wir parsen die EML-Dateien, extrahieren den Body-Text und (bei Trainingsdaten) die Kategorie (aus dem Header `X-Category`).


In [4]:
def parse_eml_file(filepath):
    """
    Parst eine EML-Datei und gibt den reinen Textinhalt zurück sowie ggf. die Kategorie.
    """
    with open(filepath, "rb") as f:
        msg = BytesParser(policy=policy.default).parse(f)
        body = msg.get_body(preferencelist=("plain",)).get_content().strip()
        return body, msg.get("X-Category", "")

# Trainingsdaten einlesen
train_data = []
for filepath in glob.glob(os.path.join(train_dir, "*.eml")):
    text, category = parse_eml_file(filepath)
    if not category:
        raise ValueError(f"Kategorie fehlt in Trainingsdatei {filepath}")
    train_data.append({"text": text, "category": category})
df_train = pd.DataFrame(train_data)
display_df(df_train, "Trainingsdaten (Text + Kategorie)")

# Testdaten einlesen
test_data = []
for filepath in glob.glob(os.path.join(test_dir, "*.eml")):
    text, category = parse_eml_file(filepath)
    if not category:
        raise ValueError(f"Kategorie fehlt in Trainingsdatei {filepath}")
    test_data.append({"text": text, "category": category})
df_test = pd.DataFrame(test_data)
display_df(df_test, "Trainingsdaten (Text + Kategorie)")



## 4. Klassifikationsfunktionen

Wir definieren zwei Funktionen:
- `classify_ticket_zero_shot`: Zero-Shot-Ansatz
- `classify_ticket_few_shot`: Few-Shot-Ansatz mit 3 Beispielen aus den Trainingsdaten


In [5]:
def classify_ticket_zero_shot(ticket_text: str) -> str:
    """
    Zero-Shot-Klassifikation: Nur Instruktion + Ticket-Text.
    """
    prompt = f"""
Du bist ein Support-System für industrielle Sensoren. Ordne das folgende Ticket einer der Kategorien zu: Installation, Störung, Lieferung.
Beachte: falls es sich um eine Störung bei der  Installation handelt, ordne es der Kategorie "Installation" zu. 
Gib nur die Kategorie zurück.

Ticket:
{ticket_text}
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Modell kann angepasst werden
        messages=[
            {"role": "system", "content": "Du bist ein hilfreiches Klassifikationssystem."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,  # Für deterministische Ausgabe
    )
    return response.choices[0].message.content.strip()

def classify_ticket_few_shot(ticket_text: str) -> str:
    """
    Few-Shot-Klassifikation: 3 feste Beispiele + Instruktion + Ticket-Text.
    """
    prompt = f"""
Du bist ein Support-System für industrielle Sensoren. Anhand der folgenden Beispiele sieh dir an, wie Tickets klassifiziert werden:

Beispiel 1:
Ticket: Sehr geehrte Damen und Herren,

unser Sensor ist nach dem Transport mit der Post angekommen und zeigt während der Installation plötzlich Abweichungen von bis zu 15% an. Wir vermuten, dass das Gerät beim Versand verstellt wurde. Benötigen Sie das Gerät
zurück oder können wir eine Fernkalibrierung durchführen? Welche Schritte schlagen Sie vor?

Freundliche Grüße,
Herr Koch

Kategorie: Installation

Beispiel 2:
Ticket: Sehr geehrtes Support-Team,

unser Drucksensor in der Verpackungslinie driftet aktuell um mehr als 13
%. Die automatische Selbstkalibrierung scheint fehlerhaft zu arbeiten. Wir haben bereits die Kalibrierungsroutine manuell durchgeführt, jedoch ohne Erfolg. In der CSV-Datei im Anhang sehen Sie die aufgezeichneten Messwerte und
 Soll-Ist-Abweichungen.

Bitte teilen Sie uns die korrekten Kalibrierungsschritte mit.

Vielen Dank,
Herr Weber

Störung

Beispiel 3:
Ticket: Hallo,

wir haben bei Ihnen die Sensoren Xi3 und X544 bestellt. Meine Bestellnummer lautet: 548268458.
Obwohl Sie gestern hätten ankommen müssen, sind sie noch nicht eingetroffen. Geben Sie mir bitte eine Rückmeldung. Wir benötigen die Sensoren dringend.

LG Herr Mustermann

Lieferung

Ordne nun das folgende Ticket einer der Kategorien zu: Installation, Störung, Lieferung.
Beachte: falls es sich um eine Störung bei der  Installation handelt, ordne es der Kategorie "Installation" zu.  Ordne Defekte, die durch einen Lieferungstransport entstanden sind, der Kategorie "Lieferung" zu.

Gebe ausschließlich den Kategorienamen zurück.

Ticket:
{ticket_text}
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Du bist ein hilfreiches Klassifikationssystem."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
    )
    return response.choices[0].message.content.strip()


## 5. Evaluation auf Daten

Wir messen, wie oft die Zero-Shot- und Few-Shot-Ansätze die korrekte Kategorie auf den Trainingsdaten vorhersagen.


In [6]:
# Zero-Shot auf Trainingsdaten
df_train["pred_zero_shot"] = df_train["text"].apply(classify_ticket_zero_shot)
df_train["correct_zero_shot"] = df_train["pred_zero_shot"] == df_train["category"]

# Few-Shot auf Trainingsdaten
df_train["pred_few_shot"] = df_train["text"].apply(classify_ticket_few_shot)
df_train["correct_few_shot"] = df_train["pred_few_shot"] == df_train["category"]

# Ergebnisse anzeigen
display_df(df_train, "Evaluation Trainingsdaten")

# Genauigkeit berechnen
acc_zero = df_train["correct_zero_shot"].mean()
acc_few = df_train["correct_few_shot"].mean()
print(f"Zero-Shot Genauigkeit auf Trainingsdaten: {acc_zero:.2%}")
print(f"Few-Shot Genauigkeit auf Trainingsdaten: {acc_few:.2%}")

Zero-Shot Genauigkeit auf Trainingsdaten: 95.00%
Few-Shot Genauigkeit auf Trainingsdaten: 90.00%


---
### Hinweise für Studierende
- Prüft, ob die Verzeichnisse `data/training_data` und `data/test_data` richtig befüllt sind.
- Passt ggf. die `temperature` an.
- Experimentiert mit mehr oder anderen Prompt-Beispielen 
- Dokumentiert eure Ergebnisse und analysiert, bei welchen Ticket-Arten die Klassifikation gut bzw. schlecht funktioniert.
